In [364]:
import numpy as np
from scipy.special import owens_t
import matplotlib.pyplot as plt
# get a logger
import logging
logger = logging.getLogger(__name__)
from ERM import fair_adversarial_error_erm
from state_evolution import OverlapSet, fair_adversarial_error_overlaps
%load_ext autoreload
%autoreload 2
from scipy.integrate import quad, dblquad

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [374]:
from data_model import KFeaturesModel
d = 1000
idx_0 = 0
idx_2 = 700
tau = 0
feature_ratios = np.array([0.5,0.5])
features_x = np.array([5,1])
features_theta = np.array([1,1])
data_model = KFeaturesModel(d, logger, delete_existing=True, normalize_matrices=True,source_pickle_path="",feature_ratios=feature_ratios,features_x=features_x,features_theta=features_theta)
# data_model.Sigma_x = data_model.Sigma_x / np.trace(data_model.Sigma_x) * d
spec_Sigma_x = np.linalg.eigvals(data_model.Sigma_x)
data_model.rho = np.mean(spec_Sigma_x * data_model.theta**2) 
data_set = data_model.generate_data(100000,tau)

In [375]:
data_model.Sigma_theta

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [376]:
data_model.Sigma_x

array([[1.66666667, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.66666667, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.66666667, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.33333333, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.33333333,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.33333333]])

In [377]:
data_set.X

array([[-0.03140859, -0.59517699,  1.10811619, ..., -0.58273646,
        -0.26937393, -0.54461768],
       [ 0.85943968,  0.39133583,  0.21340422, ...,  0.09694138,
        -0.71540863,  0.42064807],
       [ 0.75168628,  2.05204309, -1.00165427, ..., -0.4134435 ,
        -0.4205976 , -0.16374371],
       ...,
       [-0.5278573 , -0.18548004, -2.06440779, ..., -0.77563134,
        -0.49243654, -0.79743468],
       [ 0.79718241, -1.76203604,  2.50416583, ...,  0.31277689,
         1.49971894, -0.94249246],
       [ 0.95488283,  2.21476802, -0.94796962, ..., -0.31363436,
         0.03944832, -0.91545142]])

In [378]:
data_set.theta

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [379]:
# let's compute the average value of the first data-poitn * y and the last data-point * y
# let's extract all the first data_point from data_set.X
first_data_points = data_set.X[:,idx_0]
second_data_points = data_set.X[:,idx_2]

first_mean = np.mean( first_data_points * data_set.y )
second_mean = np.mean( second_data_points * data_set.y )

print(first_mean)
print(second_mean)

0.046814238868614445
0.007676663553179585


In [380]:

def get_usefulness(idx):
    return np.sqrt(2)* (data_set.theta.dot(data_model.Sigma_x))[idx]/( np.sqrt(np.pi *d )* np.sqrt(data_model.rho + tau**2) )

print(get_usefulness(idx_0))
print(get_usefulness(idx_2))


0.042052208700336005
0.0084104417400672


In [381]:
# def integrand(a,b, idx):
#     v = np.array([a,b])
#     va = v[0]
#     vb = v[1]
#     val = vb
#     if idx == 0:
#         val = va
#     return val*np.sign(data_set.theta.dot(v)/np.sqrt(d)) * 1/(2*np.pi*np.sqrt( np.trace(data_model.Sigma_x) ) ) * np.exp(-0.5 * v.dot(np.linalg.inv(data_model.Sigma_x)@v))

# I0 = dblquad(lambda x,y: integrand(x,y,0), -np.inf, np.inf, lambda x: -np.inf, lambda x: np.inf)
# I1 = dblquad(lambda x,y: integrand(x,y,1), -np.inf, np.inf, lambda x: -np.inf, lambda x: np.inf)


In [373]:
# print(I0)
# print(I1)